In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV,cross_val_score
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score


In [2]:
selected_features=['partner',
 'onlinebackup',
 'tenure',
 'contract',
 'monthlycharges',
 'totalcharges',
 'techsupport', 'internetservice',
 'multiplelines']

In [3]:
x_train_bal=pd.read_csv("x_train_bal.csv")
y_train_bal=pd.read_csv("y_train_bal.csv")
x_train_bal=x_train_bal[selected_features]

In [4]:
x_train_bal.head()

,partner,onlinebackup,tenure,contract,monthlycharges,totalcharges,techsupport,internetservice,multiplelines
0,1,1,65,2,94.55,6078.75,1,2,1
1,0,0,26,0,35.75,1022.50,1,1,2
2,1,1,68,2,90.20,6297.65,1,2,1
3,0,1,3,0,84.30,235.05,0,2,0
4,1,0,49,0,40.65,2070.75,0,1,2


In [5]:
y_train_bal.value_counts()

churn
0        4130
1        4130
Name: count, dtype: int64

In [6]:
param_grid ={
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True],
    'max_leaf_nodes': [None, 10, 20],
    'min_impurity_decrease': [0.0, 0.01],
    'class_weight': ['balanced'],
    'warm_start': [False, True],
    'verbose': [0]
}

In [ ]:
grid_skf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=3,               # <-- use StratifiedKFold here
    scoring='accuracy',
    verbose=3,
    n_jobs=-1 
)

In [10]:
grid_skf.fit(x_train_bal, y_train_bal)
print("Best params (StratifiedKFold):", grid_skf.best_params_)

Fitting 3 folds for each of 1728 candidates, totalling 5184 fits


c:\Users\PUNEETH\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best params (StratifiedKFold): {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100, 'verbose': 0, 'warm_start': False}


 {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100, 'verbose': 0, 'warm_start': False}

In [11]:
grid_skf.best_score_

np.float64(0.8124785505005301)

In [16]:
grid_skf2 = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,               # <-- use StratifiedKFold here
    scoring='accuracy',
    verbose=3,
    n_jobs=-1 
)
grid_skf2.fit(x_train_bal, y_train_bal)
print("Best params (StratifiedKFold):", grid_skf.best_params_)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


c:\Users\PUNEETH\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best params (StratifiedKFold): {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 20, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200, 'verbose': 0, 'warm_start': False}


In [18]:
grid_skf2.best_score_

np.float64(0.82409200968523)

In [21]:
grid_skf.best_params_

{'bootstrap': True,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 20,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 200,
 'verbose': 0,
 'warm_start': False}

In [23]:
rf_model = RandomForestClassifier(
    bootstrap=True,
    class_weight='balanced',
    criterion='gini',
    max_depth=20,
    max_features='sqrt',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_samples_leaf=2,
    min_samples_split=2,
    n_estimators=200,
    verbose=0,
    warm_start=False
)

In [24]:
rf_model.fit(x_train_bal, y_train_bal)

c:\Users\PUNEETH\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(class_weight='balanced', max_depth=20,
                       min_samples_leaf=2, n_estimators=200)

In [ ]:
x_test=pd.read_csv("x_test.csv")
y_test=pd.read_csv("y_test.csv")
x_test=x_test[selected_features]

In [26]:
y_pred_rf = rf_model.predict(x_test)
y_prob_rf = rf_model.predict_proba(x_test)[:, 1]

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- dependents
- deviceprotection
- gender
- onlinesecurity
- paperlessbilling
- ...
